In [1]:
!cd ~/Developer/kuzu/ &&  make clean-python-api && make release NUM_THREADS=96

rm -rf tools/python_api/build
cd external && \
mkdir -p build && \
cd build && \
cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
cmake --build . --config Release -- -j 96
-- Configuring done
-- Generating done
-- Build files have been written to: /home/lc/Developer/kuzu/external/build
gmake[1]: Entering directory '/home/lc/Developer/kuzu/external/build'
gmake[2]: Entering directory '/home/lc/Developer/kuzu/external/build'
gmake[3]: Entering directory '/home/lc/Developer/kuzu/external/build'
gmake[3]: Leaving directory '/home/lc/Developer/kuzu/external/build'
[100%] Built target apache_arrow
gmake[2]: Leaving directory '/home/lc/Developer/kuzu/external/build'
gmake[1]: Leaving directory '/home/lc/Developer/kuzu/external/build'
mkdir -p build/release && \
cd build/release && \
cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
cmake --build . --config Release -- -j 96
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /home/lc/Developer/kuzu/bui

In [2]:
import sys
sys.path.append('/home/lc/Developer/kuzu/tools/python_api/build')
import kuzu
import torch
import time
from torch_geometric.loader import NeighborLoader

In [3]:
DB_PATH = '/home/lc/Developer/mag100m-kuzu'
db = kuzu.Database(DB_PATH, 50*1024**3)
fs, gs = db.get_torch_geometric_remote_backend()

In [4]:
conn = kuzu.Connection(db)
paper_count = conn.execute("MATCH (p:Paper) RETURN count(p) AS count").get_next()[0]

In [5]:
paper_count

111059956

In [6]:
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [30] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=1024,
    input_nodes=('Paper', torch.arange(0, paper_count)),
    input_time=None
)

Getting edge list size
Size: 1615685872
Executing query: MATCH (a:Paper)-[Cites]->(b:Paper) RETURN offset(id(a)), offset(id(b))
Allocating edge list
Getting results
10000000/1615685872
20000000/1615685872
30000000/1615685872
40000000/1615685872
50000000/1615685872
60000000/1615685872


In [11]:
import gc
i = 1
for b in loader_kuzu:
    print(i)
    i += 1
    del b
    print(gc.collect())
    if i == 10:
        break
start = time.time()
i = 1
for b in loader_kuzu:
    print(i)
    i += 1
    del b
    print(gc.collect())
    if i == 10000:
        break
end = time.time() - start

1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
0
22
0
23
0
24
0
25
0
26
0
27
0
28
0
29
0
30
0
31
0
32
0
33
0
34
0
35
0
36
0
37
0
38
0
39
0
40
0
41
0
42
0
43
0
44
0
45
0
46
0
47
0
48
0
49
0
50
0
51
0
52
0
53
0
54
0
55
0
56
0
57
0
58
0
59
0
60
0
61
0
62
0
63
0
64
0
65
0
66
0
67
0
68
0
69
0
70
0
71
0
72
0
73
0
74
0
75
0
76
0
77
0
78
0
79
0
80
0
81
0
82
0
83
0
84
0
85
0
86
0
87
0
88
0
89
0
90
0
91
0
92
0
93
0
94
0
95
0
96
0
97
0
98
0
99
0
100
0
101
0
102
0
103
0
104
0
105
0
106
0
107
0
108
0
109
0
110
0
111
0
112
0
113
0
114
0
115
0
116
0
117
0
118
0
119
0
120
0
121
0
122
0
123
0
124
0
125
0
126
0
127
0
128
0
129
0
130
0
131
0
132
0
133
0
134
0
135
0
136
0
137
0
138
0
139
0
140
0
141
0
142
0
143
0
144
0
145
0
146
0
147
0
148
0
149
0
150
0
151
0
152
0
153
0
154
0
155
0
156
0
157
0
158
0
159
0
160
0
161
0
162
0
163
0
164
0
165
0
166
0
167
0
168
0
169
0
170
0
171
0
172
0
173
0
174
0
175
0
176
0
177
0
178
0
179


In [10]:
gc.collect()

0

In [7]:
i, end

(10, 1.8295352458953857)

In [8]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
import torch
from torch_geometric.loader import NeighborLoader
import time

dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [9]:
loader_pyg = NeighborLoader(
    data=data,
    num_neighbors={key: [30] * 2 for key in data.edge_types},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, 736389)),
    input_time=None,
    num_workers=20,
    filter_per_worker=True,
    persistent_workers=True
)

In [10]:
i = 1
for b in loader_pyg:
    i += 1
    if i == 100:
        break
start = time.time()
i = 1
for b in loader_pyg:
    i += 1
    if i == 100:
        break
end = time.time() - start

/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = value.storage()._new_shared(numel)
/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = value.storage()._new_shared(numel)
/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will b

In [11]:
i, end

(100, 0.8017268180847168)